#  Smartphone Data Cleaning and Exploration.
Generates: "cleaned(ver 1).csv"


In [ ]:
# 📦 Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits import mplot3d

# 📁 Mount Google Drive (if using Google Colab)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# -----------------------------
# 📌 Load Raw Data
# -----------------------------
columns_all = ['Quarter', 'Year', 'Brand', 'OS Version', 'Screen Size', 'Storage (GB)', 'Segment Group', 'Units', 'Value (US$M)']
columns_segment = ['Quarter', 'Year', 'Brand', 'OS Version', 'Screen Size', 'Storage (GB)', 'Units', 'Value (US$M)']

df_raw = pd.read_csv('/content/drive/MyDrive/IDC data/H_Data_smartphone.csv')
df_segment = pd.read_csv('/content/drive/MyDrive/IDC data/H_data(segment).csv')

In [ ]:
# -----------------------------
# 📌 Initial Filtering and Feature Engineering
# -----------------------------
df_clean = df_raw[df_raw['Model Name'].notna()].copy()
df_clean['ASP'] = df_clean['Value (US$M)'] / df_clean['Units']
df_clean['Share (%)'] = (df_clean['Units'] / df_clean['Units'].sum()) * 100

In [ ]:
# -----------------------------
# 📌 Cellular Technology Mapping
# -----------------------------
connectivity_map = {
    'WiFi / 2G': 2,
    'WiFi / 3G': 3,
    'WiFi / 4G': 4
}
df_clean['Cellular'] = df_clean['Connectivity'].map(connectivity_map).fillna(5).astype(int)

In [ ]:
# -----------------------------
# 📊 Brand Filtering for Graphs
# -----------------------------
top_brands = ['Apple', 'Samsung', 'Motorola', 'Sony', 'Microsoft', 'Google']
df_top = df_clean[df_clean['Brand'].isin(top_brands)].copy()

In [ ]:
# -----------------------------
# 📊 Count and Share Aggregation
# -----------------------------
df_count = df_clean.groupby(['Year', 'Brand', 'Cellular', 'Model Name']).size().reset_index(name='Model Count')
df_share = df_clean.groupby(['Year', 'Brand'])['Share (%)'].sum().reset_index()
df_stats = pd.merge(df_count, df_share, on=['Year', 'Brand'])

In [ ]:
# -----------------------------
# 📈 3D Plot: Year vs Cellular vs Brand (Apple and Samsung)
# -----------------------------
graph_data = df_stats[df_stats['Brand'].isin(['Apple', 'Samsung'])].copy()
graph_data['Brand Code'] = graph_data['Brand'].map({'Apple': 1, 'Samsung': 2})

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(
    graph_data['Year'].astype(int),
    graph_data['Cellular'],
    graph_data['Brand Code']
)
plt.title("3D View: Year vs Cellular Tech vs Brand (Apple/Samsung)")
plt.show()

In [ ]:
# -----------------------------
# 📉 Scatterplot: Year vs Model Count by Cellular
# -----------------------------
sns.relplot(
    data=graph_data,
    x="Year",
    y="Model Count",
    hue="Cellular",
    kind="scatter"
).fig.axes[0].invert_yaxis()

In [ ]:
# -----------------------------
# 📈 Line Plot: Brand Share over Time
# -----------------------------
sns.relplot(
    data=df_top,
    x="Year",
    y="Share (%)",
    hue="Brand",
    kind="line"
).fig.set_size_inches(10, 6)

In [ ]:
# -----------------------------
# 🔍 Deep Dive: Apple in 2017Q1
# -----------------------------
apple_q1 = df_raw[(df_raw['Brand'] == "Apple") & (df_raw['Year'] == 2017) & (df_raw['Quarter'] == '2017Q1')]
apple_segment_q1 = df_segment[(df_segment['Brand'] == "Apple") & (df_segment['Year'] == 2017) & (df_segment['Quarter'] == '2017Q1')]

In [ ]:
# -----------------------------
# 🔗 Merge Segment Data with Product Data
# -----------------------------
merge_cols = ['Quarter', 'Brand', 'OS Version', 'Screen Size', 'Storage (GB)']
df_model_subset = df_clean[merge_cols + ['Model Name']]
df_segment_full = df_segment[
    merge_cols + ['Year', 'RAM (GB)', 'Units', 'Value (US$M)', 'Processor Speed Band', 'Segment Group']
]

df_merged = df_segment_full.merge(df_model_subset, how='inner', on=merge_cols)
df_merged = df_merged[df_merged['Segment Group'] == 'Consumer'].drop_duplicates()

In [ ]:
# -----------------------------
# 💰 Compute ASP again
# -----------------------------
df_merged['ASP'] = (df_merged['Value (US$M)'] * 1_000_000) / df_merged['Units']
brand_revenue = df_merged.groupby('Brand')['Value (US$M)'].sum()
top_models = df_merged.nlargest(3000, 'Value (US$M)')

In [ ]:
# -----------------------------
# 📊 Descriptive Statistics
# -----------------------------
selected_brands = [
    'Apple', 'Samsung', 'Motorola', 'HTC', 'Google', 'LG Electronics',
    'Alcatel', 'BlackBerry', 'T-Mobile', 'AT&T', 'Kyocera', 'OnePlus',
    'ZTE', 'Huawei', 'Pantech', 'Nokia', 'HP', 'Microsoft', 'Sony Ericsson',
    'Palm', 'Sony', 'Coolpad', 'Sprint', 'Sanyo', 'Red'
]
df_merged[df_merged['Brand'].isin(selected_brands)].describe()

print("Overall Units Summary:\n", df_clean['Units'].describe())
print("\nAverage Storage (GB):", df_merged['Storage (GB)'].mean())

In [ ]:
# -----------------------------
# 🧹 (Optional) Reload Cleaned Data
# -----------------------------
df_final = pd.read_csv('/content/drive/MyDrive/OUTPUT DATA(FROM IDC)/cleaned(ver 1).csv')
# Example fix for incorrect syntax:
# bl = ['Apple', 'Samsung']
# df_final[df_final['Brand'].isin(bl)]